<a href="https://colab.research.google.com/github/Ashish-Soni08/Playground/blob/main/haystack/Advent_of_Haystack_Branch_based_on_language(Ashish_Soni).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advent of Haystack - Day 4
_Make a copy of this Colab to start!_


In this challenge, your task is to route documents to a specific document store based on the language of the document!

We've already started a pipeline that adds French documents to a French `InMemoryDocumentStore`, and English ones to another.

However, the dummy documents here include hotel reviews in German, Spanish, French, Dutch, and English.

👉 Your task is to complete the **Write Documents to `InMemoryDocumentStore`** section by adding branches to the pipeline. These branches should index documents to a document stores based on language.

👉 **Optionally**, you can also add a section at the end that does question answering where the answer to the question is generated from the documents with the language the question was asked in. For this section, you should use the [`TextLanguageRouter`](https://docs.haystack.deepset.ai/v2.0/docs/textlanguagerouter)

**Useful Components:**

- [`DocumentLanguageClassifier`](https://docs.haystack.deepset.ai/v2.0/docs/documentlanguageclassifier): Use this component to classify documents based on language. The language will be added to the documents `metadata`
- [`MetadataRouter`](https://docs.haystack.deepset.ai/v2.0/docs/metadatarouter): Use this components to route documents to different document stores based on metadata. In this challenge, you will set it to be routed based on language.

#Installation
**Note:** There is a known issue with colab due to a version conflict error related to `llmx` which comes with Colab. You might get an `llmx` error. You can safely ignore this, or run `pip uninstall -y llmx`

In [1]:
%%capture

!pip install haystack-ai
!pip install langdetect
!pip install transformers[torch,sentencepiece]==4.32.1 sentence-transformers>=2.2.0

## Write Documents Into `InMemoryDocumentStore`

The following indexing pipeline writes French and English documents into their own `InMemoryDocuementStores` based on langauge. Add more branches to this pipeline so that you can have a separate document store for every language 👇

In [2]:
from haystack import Document, Pipeline
from haystack.document_stores import InMemoryDocumentStore
from haystack.components.classifiers import DocumentLanguageClassifier
from haystack.components.routers import MetadataRouter
from haystack.components.writers import DocumentWriter


documents = [Document(content = "Durchgängig Lärm, weil direkt an der Partymeile; schmutziges Geschirr; unvollständige Küchenausstattung; Abzugshaube über Herd ging für zwei Stunden automatisch an und lies sich nicht abstellen; Reaktionen auf Anfragen entweder gar nicht oder unfreundlich"),
             Document(content = "Das Personal ist sehr zuvorkommend! Über WhatsApp war man im guten Kontakt und konnte alles erfragen. Auch das Angebot des Shuttleservices war super und würde ich empfehlen - sehr unkompliziert! Unser Flug hatte Verspätung und der Shuttle hat auf uns gewartet. Die Lage zur Innenstadt ist sehr gut,jedoch ist die Fensterfront direkt zur Club-Straße deshalb war es nachts bis drei/vier Uhr immer recht laut. Die Kaffeemaschine oder auch die Couch hätten sauberer sein können. Ansonsten war das Appartement aber völlig ok."),
             Document(content = "Super appartement. Juste au dessus de plusieurs bars qui ferment très tard. A savoir à l'avance. (Bouchons d'oreilles fournis !)"),
             Document(content = "El apartamento estaba genial y muy céntrico, todo a mano. Al lado de la librería Lello y De la Torre de los clérigos. Está situado en una zona de marcha, así que si vais en fin de semana , habrá ruido, aunque a nosotros no nos molestaba para dormir"),
             Document(content = "The keypad with a code is convenient and the location is convenient. Basically everything else, very noisy, wi-fi didn't work, check-in person didn't explain anything about facilities, shower head was broken, there's no cleaning and everything else one may need is charged."),
             Document(content = "It is very central and appartement has a nice appearance (even though a lot IKEA stuff), *W A R N I N G** the appartement presents itself as a elegant and as a place to relax, very wrong place to relax - you cannot sleep in this appartement, even the beds are vibrating from the bass of the clubs in the same building - you get ear plugs from the hotel -> now I understand why -> I missed a trip as it was so loud and I could not hear the alarm next day due to the ear plugs.- there is a green light indicating 'emergency exit' just above the bed, which shines very bright at night - during the arrival process, you felt the urge of the agent to leave as soon as possible. - try to go to 'RVA clerigos appartements' -> same price, super quiet, beautiful, city center and very nice staff (not an agency)- you are basically sleeping next to the fridge, which makes a lot of noise, when the compressor is running -> had to switch it off - but then had no cool food and drinks. - the bed was somehow broken down - the wooden part behind the bed was almost falling appart and some hooks were broken before- when the neighbour room is cooking you hear the fan very loud. I initially thought that I somehow activated the kitchen fan"),
             Document(content = "Un peu salé surtout le sol. Manque de service et de souplesse"),
             Document(content = "De comfort zo centraal voor die prijs."),
             Document(content = "Die Lage war sehr Zentral und man konnte alles sehenswertes zu Fuß erreichen. Wer am Wochenende nachts schlafen möchte, sollte diese Unterkunft auf keinen Fall nehmen. Party direkt vor der Tür so das man denkt, man schläft mitten drin. Sehr Sehr laut also und das bis früh 5 Uhr. Ab 7 kommt dann die Straßenreinigung die keineswegs leiser ist."),
             Document(content = "Nous avons passé un séjour formidable. Merci aux personnes , le bonjours à Ricardo notre taxi man, très sympathique. Je pense refaire un séjour parmi vous, après le confinement, tout était parfait, surtout leur gentillesse, aucune chaude négative. Je n'ai rien à redire de négative, Ils étaient a notre écoute, un gentil message tout les matins, pour nous demander si nous avions besoins de renseignement et savoir si tout allait bien pendant notre séjour."),
             Document(content = "Céntrico. Muy cómodo para moverse y ver Oporto. Edificio con terraza propia en la última planta. Todo reformado y nuevo. Te traen un estupendo desayuno todas las mañanas al apartamento. Solo que se puede escuchar algo de ruido de la calle a primeras horas de la noche. Es un zona de ocio nocturno. Pero respetan los horarios.")
]

fr_document_store = InMemoryDocumentStore()
en_document_store = InMemoryDocumentStore()
de_document_store = InMemoryDocumentStore()
es_document_store = InMemoryDocumentStore()
nl_document_store = InMemoryDocumentStore()


language_classifier = DocumentLanguageClassifier(languages = ["en", "fr", "de", "es", "nl"])
router_rules = {
                "en": {"language": {"$eq": "en"}},
                "fr": {"language": {"$eq": "fr"}},
                "de": {"language": {"$eq": "de"}},
                "es": {"language": {"$eq": "es"}},
                "nl": {"language": {"$eq": "nl"}}
                }
router = MetadataRouter(rules=router_rules)

fr_writer = DocumentWriter(document_store = fr_document_store)
en_writer = DocumentWriter(document_store = en_document_store)
de_writer = DocumentWriter(document_store = de_document_store)
es_writer = DocumentWriter(document_store = es_document_store)
nl_writer = DocumentWriter(document_store = nl_document_store)

indexing_pipeline = Pipeline()
indexing_pipeline.add_component(instance=language_classifier, name="language_classifier")
indexing_pipeline.add_component(instance=router, name="router")
indexing_pipeline.add_component(instance=fr_writer, name="fr_writer")
indexing_pipeline.add_component(instance=en_writer, name="en_writer")
indexing_pipeline.add_component(instance=de_writer, name="de_writer")
indexing_pipeline.add_component(instance=es_writer, name="es_writer")
indexing_pipeline.add_component(instance=nl_writer, name="nl_writer")


indexing_pipeline.connect("language_classifier", "router")
indexing_pipeline.connect("router.en", "en_writer")
indexing_pipeline.connect("router.fr", "fr_writer")
indexing_pipeline.connect("router.de", "de_writer")
indexing_pipeline.connect("router.es", "es_writer")
indexing_pipeline.connect("router.nl", "nl_writer")

In [3]:
indexing_pipeline.run(data={"language_classifier": {"documents": documents}})

{'router': {'unmatched': []},
 'en_writer': {'documents_written': 2},
 'fr_writer': {'documents_written': 3},
 'de_writer': {'documents_written': 3},
 'es_writer': {'documents_written': 2},
 'nl_writer': {'documents_written': 1}}

## Check the Contents of Your Document Stores

You can check the contents of your document stores 👇

In [10]:
len(documents)

11

In [4]:
en_document_store.filter_documents()

[Document(id=8f64ab234c6a5d5652d02bed144d069ec6e988903b071d16fffbf400abfc1047, content: 'The keypad with a code is convenient and the location is convenient. Basically everything else, very...', meta: {'language': 'en'}),
 Document(id=d4d878288efba5e28a43ae0195e43dadd0298fe36d3d9b3075c5c5120d27763e, content: 'It is very central and appartement has a nice appearance (even though a lot IKEA stuff), *W A R N I ...', meta: {'language': 'en'})]

In [5]:
fr_document_store.filter_documents()

[Document(id=ea7ea338874232de2d8105a258813f50345db82772e21ad2c4549dbb7adce8a3, content: 'Super appartement. Juste au dessus de plusieurs bars qui ferment très tard. A savoir à l'avance. (Bo...', meta: {'language': 'fr'}),
 Document(id=6b64c8a60543ee32b81cd39bc8d6e09fae4bff1b22c6ccdcf414db26fa354e7a, content: 'Un peu salé surtout le sol. Manque de service et de souplesse', meta: {'language': 'fr'}),
 Document(id=b1be23526f19a8af80a190e775bfd05e65878e585529037cb45b47267a4eaa98, content: 'Nous avons passé un séjour formidable. Merci aux personnes , le bonjours à Ricardo notre taxi man, t...', meta: {'language': 'fr'})]

In [6]:
de_document_store.filter_documents()

[Document(id=d565bf734fac0a7e8eebfde8f67ec7f98c485d4b8c3e959249e272dabdbd51a9, content: 'Durchgängig Lärm, weil direkt an der Partymeile; schmutziges Geschirr; unvollständige Küchenausstatt...', meta: {'language': 'de'}),
 Document(id=3f34d6e83d3ddf1ddf811efe2ad8a506906353a2a7f5bbf8d4e90ef5f173d693, content: 'Das Personal ist sehr zuvorkommend! Über WhatsApp war man im guten Kontakt und konnte alles erfragen...', meta: {'language': 'de'}),
 Document(id=89a1da2e53f3987f9cd79405223b118b44beff2941d345bce86dd2502398fe72, content: 'Die Lage war sehr Zentral und man konnte alles sehenswertes zu Fuß erreichen. Wer am Wochenende nach...', meta: {'language': 'de'})]

In [7]:
es_document_store.filter_documents()

[Document(id=72b094c163b22a660528bc5adbdf0fecf96b4b4d753c1b117f15dba482d2f948, content: 'El apartamento estaba genial y muy céntrico, todo a mano. Al lado de la librería Lello y De la Torre...', meta: {'language': 'es'}),
 Document(id=4b37b8bdfffccfb3211ea167b4fdc5121ca51fc5f869b4f834e8da473f0d3353, content: 'Céntrico. Muy cómodo para moverse y ver Oporto. Edificio con terraza propia en la última planta. Tod...', meta: {'language': 'es'})]

In [8]:
nl_document_store.filter_documents()

[Document(id=301e7c12f68ff64e23c23841911ad537199d697318ea2f4ca3fb9289b3cec5d0, content: 'De comfort zo centraal voor die prijs.', meta: {'language': 'nl'})]